# Instruções gerais

Para executar uma CÉLULA, pressione o botão "Run" no topo, com a CÉLULA selecionada, ou pressione:

```
Shift + Enter
```

EXECUTE APENAS UMA CÉLULA POR VEZ

\-Caso deseje apagar a saída de uma célula carregada, mas não deseje carregar uma nova saída, vá à aba superior, escolha o ícone do teclado (open the command palette/ jupyter-notebook command group) e selecione clear cell output.

@author: Marco César Prado Soares, MSc. Especialista Lean Six Sigma Master Black Belt, Eng. Químico, MSc. Eng. Mecatrônica (instrumentação) [Marco.Soares@br.ey.com](mailto:&#77;&#97;&#x72;&#x63;&#111;&#46;&#x53;&#111;&#97;&#114;&#101;&#x73;&#x40;&#98;&#x72;&#x2e;&#x65;&#121;&#46;&#99;&#111;&#109;); [marcosoares.feq@gmail.com](mailto:&#x6d;&#x61;&#114;&#99;&#111;&#115;&#x6f;&#97;&#114;&#101;&#x73;&#x2e;&#x66;&#101;&#113;&#64;&#103;&#109;&#97;&#x69;&#x6c;&#46;&#x63;&#x6f;&#109;)

**Executar múltiplas tabelas de um arquivo Excel**

**Exemplo 1**

Dados podem ser obtidos em: https://www.youtube.com/watch?v=nBu1Bqa1jjs&t=1780s

In [ ]:
import pandas as pd
filename = "Data_files/016-MSPTDA-Excel.xlsx"
df = pd.read_excel(filename, sheet_name = "Tables", engine='openpyxl')
#Note que o nome da tabela é fornecido em sheet_name

#view the first five rows: 
df.head()

**Exemplo 2**

<span style="color: rgb(81, 81, 81); font-family: &quot;Segoe UI&quot;, SegoeUI, Roboto, &quot;Segoe WP&quot;, &quot;Helvetica Neue&quot;, Helvetica, Tahoma, Arial, sans-serif; font-size: 20px; background-color: rgb(255, 255, 255);">The better way is via&nbsp;</span> [Openpyxl](https://openpyxl.readthedocs.io/en/stable/usage.html)<span style="color: rgb(81, 81, 81); font-family: &quot;Segoe UI&quot;, SegoeUI, Roboto, &quot;Segoe WP&quot;, &quot;Helvetica Neue&quot;, Helvetica, Tahoma, Arial, sans-serif; font-size: 20px; background-color: rgb(255, 255, 255);">, a python module dedicated to working with Excel files. It has a method -&nbsp;</span> `._tables` <span style="color: rgb(81, 81, 81); font-family: &quot;Segoe UI&quot;, SegoeUI, Roboto, &quot;Segoe WP&quot;, &quot;Helvetica Neue&quot;, Helvetica, Tahoma, Arial, sans-serif; font-size: 20px; background-color: rgb(255, 255, 255);">&nbsp;that allows access to defined tables in the spreadsheet.</span>

In [ ]:
#import library
from openpyxl import load_workbook

#read file
wb = load_workbook(filename)

#access specific sheet
ws = wb["Tables"]

We can access the tables in the worksheet through the `tables` method - this returns a dictionary :

```
{key : value for key, value in ws.tables.items()}

```

```
{'dSalesReps': 'A1:C26',
 'dProduct': 'E1:I17',
 'dCategory': 'K1:L6',
 'dSupplier': 'N1:R5'}
```

From the result above, we can see the name of each table (_name=dSalesReps_) and the span of the data (_ref='A1:A26_). Let's get our data out :

In [ ]:
{key : value for key, value in ws.tables.items()}
mapping = {}

for entry, data_boundary in ws.tables.items():
    #parse the data within the ref boundary
    data = ws[data_boundary]
    #extract the data 
    #the inner list comprehension gets the values for each cell in the table
    content = [[cell.value for cell in ent] 
               for ent in data
          ]
    
    header = content[0]
    
    #the contents ... excluding the header
    rest = content[1:]
    
    #create dataframe with the column names
    #and pair table name with dataframe
    df = pd.DataFrame(rest, columns = header)
    mapping[entry] = df

mapping

We can safely extract our tables <span style="color: #af00db;">from</span> the dictionary :

In [ ]:
dSalesReps, dProduct, dCategory, dSupplier = mapping.values()

<span style="color: rgb(81, 81, 81); font-family: &quot;Segoe UI&quot;, SegoeUI, Roboto, &quot;Segoe WP&quot;, &quot;Helvetica Neue&quot;, Helvetica, Tahoma, Arial, sans-serif; font-size: 20px; background-color: rgb(255, 255, 255);">Let's view some of the dataframes</span>

In [ ]:
dSalesReps.head()

<span style="font-family: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; font-size: 15px;"><b>Exemplo 1</b></span>

<span style="font-family: inherit; font-style: inherit; font-variant-ligatures: inherit; font-variant-caps: inherit; font-weight: inherit; font-size: 15px;">I wrote the following code to identify the multiple tables automatically, in case you have many files you need to process and don't want to look in each one to get the right row numbers. The code also looks for non-empty rows above each table and reads those as table metadata.</span>

In [ ]:
def parse_excel_sheet(file, sheet_name=0, threshold=5):
    '''parses multiple tables from an excel sheet into multiple data frame objects. Returns [dfs, df_mds], where dfs is a list of data frames and df_mds their potential associated metadata'''
    xl = pd.ExcelFile(file)
    entire_sheet = xl.parse(sheet_name=sheet_name)

    # count the number of non-Nan cells in each row and then the change in that number between adjacent rows
    n_values = np.logical_not(entire_sheet.isnull()).sum(axis=1)
    n_values_deltas = n_values[1:] - n_values[:-1].values

    # define the beginnings and ends of tables using delta in n_values
    table_beginnings = n_values_deltas > threshold
    table_beginnings = table_beginnings[table_beginnings].index
    table_endings = n_values_deltas < -threshold
    table_endings = table_endings[table_endings].index
    if len(table_beginnings) < len(table_endings) or len(table_beginnings) > len(table_endings)+1:
        raise BaseException('Could not detect equal number of beginnings and ends')

    # look for metadata before the beginnings of tables
    md_beginnings = []
    for start in table_beginnings:
        md_start = n_values.iloc[:start][n_values==0].index[-1] + 1
        md_beginnings.append(md_start)

    # make data frames
    dfs = []
    df_mds = []
    for ind in range(len(table_beginnings)):
        start = table_beginnings[ind]+1
        if ind < len(table_endings):
            stop = table_endings[ind]
        else:
            stop = entire_sheet.shape[0]
        df = xl.parse(sheet_name=sheet_name, skiprows=start, nrows=stop-start)
        dfs.append(df)

        md = xl.parse(sheet_name=sheet_name, skiprows=md_beginnings[ind], nrows=start-md_beginnings[ind]-1).dropna(axis=1)
        df_mds.append(md)
    return dfs, df_mds